# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [310]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [311]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/Users/jonathansweeney/Udacity/Data-Engineering-AWS-Nanodegree/Project-1-NoSql-DB


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [312]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [313]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

411


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [314]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [315]:
def executeCreateOrDrop(query):
    """Executes Query to Create or Drop a Table or Keyspace
        Arguments:
        ----------
        query - str - query to be executed
    """
    try:
        session.execute(query)
    except Exception as e:
        print(e)

#### Create Keyspace

In [316]:
create_keyspace_query = """
    CREATE KEYSPACE IF NOT EXISTS project1
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""

executeCreateOrDrop(create_keyspace_query)


#### Set Keyspace

In [317]:
try:
    session.set_keyspace('project1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4  

For table song_history, the column sessionId was used as a partition key because the queries will filter by this column. Column itemInSession was used as a clustering column to help make up a unique key.

In [318]:
create_song_history_query = "CREATE TABLE IF NOT EXISTS song_history (sessionId int, itemInSession int, \
        artist varchar, songTitle varchar, length float, \
        PRIMARY KEY ((sessionId), itemInSession))"

executeCreateOrDrop(create_song_history_query)

In [319]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_history (sessionId, itemInSession, songTitle, length) "
        query += "VALUES (%s, %s, %s, %s)" 
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [320]:
# Check the number or rows in the database
rows = session.execute("SELECT * FROM song_history")

for row in rows:
    print(sum(1 for row in rows))

410


In [321]:
rows = session.execute("SELECT artist, songTitle, length FROM song_history WHERE sessionId = 128 AND itemInSession=11")

for row in rows:
    print(row)

Row(artist=None, songtitle='Such Great Heights', length=266.4746398925781)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

For table session_user_library, the columns userId and sessionId were used as a partition key because the queries will filter by this columns. Column itemInSession was used as a clustering column to help make up a unique key.

In [322]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_session_user_history_query = "CREATE TABLE IF NOT EXISTS session_user_history (userId int, sessionId int, itemInSession int, \
        artist varchar, songTitle varchar, firstName varchar, lastName varchar, \
        PRIMARY KEY ((userId, sessionId), itemInSession))"

executeCreateOrDrop(create_session_user_history_query)

                

In [323]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_user_history (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName) "
        query += "VALUES (%s, %s, %s, %s, %s, %s, %s)" 
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [324]:
rows = session.execute("SELECT artist, songTitle, firstName, lastName FROM session_user_history WHERE userId = 43 and sessionId = 42")

for row in rows:
    print(row)    

Row(artist='Gary Hobbs', songtitle='En Mi Mundo', firstname='Jahiem', lastname='Miles')
Row(artist='Lifehouse', songtitle="We'll Never Know", firstname='Jahiem', lastname='Miles')
Row(artist='Olivia Ruiz', songtitle='Cabaret Blanco', firstname='Jahiem', lastname='Miles')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'  

For table song_users, the songTitle was used as a partition key because the queries will filter by this column. UserId was used as a clustering column to help make up a unique key.

In [325]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_song_user_history_query = "CREATE TABLE IF NOT EXISTS song_users (songTitle varchar, userId int, \
        artist varchar, firstName varchar, lastName varchar, \
        PRIMARY KEY ((songTitle), userId))"

executeCreateOrDrop(create_song_user_history_query)

In [326]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (userId, artist, songTitle, firstName, lastName) "
        query += "VALUES (%s, %s, %s, %s, %s)" 
        try:
            session.execute(query, (int(line[10]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

In [327]:
rows = session.execute("SELECT artist, songTitle, firstName, lastName FROM song_users WHERE songTitle='Big Munny'")

for row in rows:
    print(row)    

Row(artist='Swizz Beatz', songtitle='Big Munny', firstname='Jayden', lastname='Graves')


### Drop the tables before closing out the sessions

In [328]:
## TO-DO: Drop the table before closing out the sessions
table_names = ["music_history", "session_user_history", "song_users"]
for table_name in table_names:
    executeCreateOrDrop(f"DROP TABLE IF EXISTS {table_name}")

### Close the session and cluster connection¶

In [329]:
session.shutdown()
cluster.shutdown()